# Model Fit in Linear Regression - Lab

## Introduction
In this lab, you'll learn how to evaluate your model results and you'll learn how to select the appropriate features using stepwise selection.

## Objectives
You will be able to:
* Use stepwise selection methods to determine the most important features for a model
* Use recursive feature elimination to determine the most important features for a model

## The Ames Housing Data once more

In [1]:
import pandas as pd
import numpy as np

ames = pd.read_csv('ames.csv')

continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

ames_cont = ames[continuous]

# log features
log_names = [f'{column}_log' for column in ames_cont.columns]

ames_log = np.log(ames_cont)
ames_log.columns = log_names

# normalize (subract mean and divide by std)

def normalize(feature):
    return (feature - feature.mean()) / feature.std()

ames_log_norm = ames_log.apply(normalize)

# one hot encode categoricals
ames_ohe = pd.get_dummies(ames[categoricals], prefix=categoricals, drop_first=True)

preprocessed = pd.concat([ames_log_norm, ames_ohe], axis=1)

## Perform stepwise selection

The function for stepwise selection is copied below. Use this provided function on your preprocessed Ames Housing data.

In [2]:
import statsmodels.api as sm

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ 
    Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [3]:
preprocessed.columns

Index(['LotArea_log', '1stFlrSF_log', 'GrLivArea_log', 'SalePrice_log',
       'BldgType_2fmCon', 'BldgType_Duplex', 'BldgType_Twnhs',
       'BldgType_TwnhsE', 'KitchenQual_Fa', 'KitchenQual_Gd', 'KitchenQual_TA',
       'SaleType_CWD', 'SaleType_Con', 'SaleType_ConLD', 'SaleType_ConLI',
       'SaleType_ConLw', 'SaleType_New', 'SaleType_Oth', 'SaleType_WD',
       'MSZoning_FV', 'MSZoning_RH', 'MSZoning_RL', 'MSZoning_RM',
       'Street_Pave', 'Neighborhood_Blueste', 'Neighborhood_BrDale',
       'Neighborhood_BrkSide', 'Neighborhood_ClearCr', 'Neighborhood_CollgCr',
       'Neighborhood_Crawfor', 'Neighborhood_Edwards', 'Neighborhood_Gilbert',
       'Neighborhood_IDOTRR', 'Neighborhood_MeadowV', 'Neighborhood_Mitchel',
       'Neighborhood_NAmes', 'Neighborhood_NPkVill', 'Neighborhood_NWAmes',
       'Neighborhood_NoRidge', 'Neighborhood_NridgHt', 'Neighborhood_OldTown',
       'Neighborhood_SWISU', 'Neighborhood_Sawyer', 'Neighborhood_SawyerW',
       'Neighborhood_Somerst', 'Nei

In [4]:
# Your code here
X = preprocessed.drop('SalePrice_log', axis=1)
y = preprocessed['SalePrice_log']

result = stepwise_selection(X, y, verbose=True)
print (result)

/Users/jrchen/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2580: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


Add  GrLivArea_log                  with p-value 1.59847e-243
Add  KitchenQual_TA                 with p-value 1.56401e-67
Add  1stFlrSF_log                   with p-value 7.00069e-48
Add  KitchenQual_Fa                 with p-value 1.70471e-37
Add  Neighborhood_OldTown           with p-value 3.20105e-23
Add  KitchenQual_Gd                 with p-value 4.12635e-21
Add  Neighborhood_Edwards           with p-value 9.05184e-17
Add  Neighborhood_IDOTRR            with p-value 1.10068e-18
Add  LotArea_log                    with p-value 1.71728e-13
Add  Neighborhood_NridgHt           with p-value 7.05633e-12
Add  BldgType_Duplex                with p-value 4.30647e-11
Add  Neighborhood_NAmes             with p-value 2.25803e-09
Add  Neighborhood_SWISU             with p-value 5.40743e-09
Add  Neighborhood_BrkSide           with p-value 8.79638e-10
Add  Neighborhood_Sawyer            with p-value 6.92011e-09
Add  Neighborhood_NoRidge           with p-value 5.87105e-08
Add  Neighborhood_Somer

/Users/jrchen/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:46: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.


Add  Neighborhood_Mitchel           with p-value 0.00994666
Drop Neighborhood_Somerst           with p-value 0.0500753
Add  Neighborhood_SawyerW           with p-value 0.00427685
['GrLivArea_log', 'KitchenQual_TA', '1stFlrSF_log', 'KitchenQual_Fa', 'Neighborhood_OldTown', 'KitchenQual_Gd', 'Neighborhood_Edwards', 'Neighborhood_IDOTRR', 'LotArea_log', 'Neighborhood_NridgHt', 'BldgType_Duplex', 'Neighborhood_NAmes', 'Neighborhood_SWISU', 'Neighborhood_BrkSide', 'Neighborhood_Sawyer', 'Neighborhood_NoRidge', 'Neighborhood_StoneBr', 'Neighborhood_MeadowV', 'SaleType_New', 'Neighborhood_BrDale', 'MSZoning_RM', 'MSZoning_RL', 'MSZoning_FV', 'MSZoning_RH', 'Neighborhood_NWAmes', 'Neighborhood_Mitchel', 'Neighborhood_SawyerW']


### Build the final model again in Statsmodels

In [5]:
preprocessed.rename(columns={'1stFlrSF_log': 'firstFlrSF_log'}, inplace=True)

In [6]:
# Your code here
from statsmodels.formula.api import ols

def build_model(df, outcome):
    predictors = ' + '.join(df.drop(columns=[outcome]).columns)
    formula = outcome + ' ~ ' + predictors
    model = ols(formula=formula, data=df).fit()
    model.summary()
    return model

In [7]:
model = build_model(df=preprocessed, outcome='SalePrice_log')
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          SalePrice_log   R-squared:                       0.839
Model:                            OLS   Adj. R-squared:                  0.834
Method:                 Least Squares   F-statistic:                     156.5
Date:                Sat, 03 Oct 2020   Prob (F-statistic):               0.00
Time:                        21:06:47   Log-Likelihood:                -738.14
No. Observations:                1460   AIC:                             1572.
Df Residuals:                    1412   BIC:                             1826.
Df Model:                          47                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -0.1317      0.263     -0.500      0.617      -0.648       0.385
LotArea_log              0.1033      0.019      5.475      0.000       0.066       0.140
firstFlrSF_log           0.1371      0.016      8.584      0.000       0.106       0.168
GrLivArea_log            0.3768      0.016     24.114      0.000       0.346       0.407
BldgType_2fmCon         -0.1715      0.079     -2.173      0.030      -0.326      -0.017
BldgType_Duplex         -0.4203      0.062     -6.813      0.000      -0.541      -0.299
BldgType_Twnhs          -0.1403      0.093     -1.513      0.130      -0.322       0.042
BldgType_TwnhsE         -0.0512      0.060     -0.858      0.391      -0.168       0.066
KitchenQual_Fa          -0.9999      0.088    -11.315      0.000      -1.173      -0.827
KitchenQual_Gd          -0.3820      0.050     -7.613      0.000      -0.480      -0.284
KitchenQual_TA          -0.6692      0.055    -12.111      0.000      -0.778      -0.561
SaleType_CWD             0.2285      0.215      1.061      0.289      -0.194       0.651
SaleType_Con             0.5861      0.304      1.927      0.054      -0.010       1.183
SaleType_ConLD           0.3151      0.155      2.029      0.043       0.010       0.620
SaleType_ConLI           0.0331      0.195      0.169      0.865      -0.350       0.416
SaleType_ConLw           0.0161      0.196      0.082      0.935      -0.368       0.400
SaleType_New             0.2999      0.079      3.803      0.000       0.145       0.455
SaleType_Oth             0.1178      0.245      0.480      0.631      -0.364       0.599
SaleType_WD              0.1748      0.065      2.676      0.008       0.047       0.303
MSZoning_FV              1.0666      0.193      5.526      0.000       0.688       1.445
MSZoning_RH              0.8768      0.194      4.512      0.000       0.496       1.258
MSZoning_RL              0.9961      0.162      6.151      0.000       0.678       1.314
MSZoning_RM              1.1023      0.152      7.264      0.000       0.805       1.400
Street_Pave             -0.2131      0.180     -1.182      0.237      -0.567       0.141
Neighborhood_Blueste     0.0529      0.318      0.167      0.868      -0.571       0.677
Neighborhood_BrDale     -0.4627      0.171     -2.711      0.007      -0.798      -0.128
Neighborhood_BrkSide    -0.6498      0.137     -4.735      0.000      -0.919      -0.381
Neighborhood_ClearCr    -0.2102      0.144     -1.456      0.146      -0.493       0.073
Neighborhood_CollgCr    -0.0761      0.119     -0.641      0.522      -0.309       0.157
Neighborhood_Crawfor    -0.0823      0.129     -0.638      0.523      -0.335       0.171
Neighborhood_Edwards    -0.7613      0.124     -6.143      0.000      -1.004      -0.518
Neighborhood_Gilbert    -0.0980      0.126     -0.777    

## Use Feature ranking with recursive feature elimination

Use feature ranking to select the 5 most important features

In [8]:
# Your code here
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
selector = RFE(linreg, 5)
selector = selector.fit(X, y.values.ravel())
# convert y to 1d np array to prevent DataConversionWarning
selector.support_

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True,  True,  True,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False])

Fit the linear regression model again using the 5 selected columns

In [9]:
# Your code here
select_columns = X.columns[selector.support_]
select_df = X[select_columns]

In [10]:
linreg.fit(select_df,y)
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Now, predict $\hat y$ using your model. You can use `.predict()` in scikit-learn. 

In [11]:
# Your code here
yhat = linreg.predict(select_df)

Now, using the formulas of R-squared and adjusted R-squared below, and your Python/numpy knowledge, compute them and contrast them with the R-squared and adjusted R-squared in your statsmodels output using stepwise selection. Which of the two models would you prefer?

$SS_{residual} = \sum (y - \hat{y})^2 $

$SS_{total} = \sum (y - \bar{y})^2 $

$R^2 = 1- \dfrac{SS_{residual}}{SS_{total}}$

$R^2_{adj}= 1-(1-R^2)\dfrac{n-1}{n-p-1}$

In [12]:
SS_Residual = np.sum((y-yhat)**2)
SS_Total = np.sum((y-np.mean(y))**2)
r_squared = 1 - (float(SS_Residual))/SS_Total
adjusted_r_squared = 1 - (1-r_squared)*(len(y)-1)/(len(y)-select_df.shape[1]-1)

In [13]:
# Your code here
print ('r_squared is {}'.format(round(r_squared,6)))
print ('adjusted_r_squared is {}'.format(round(adjusted_r_squared,6)))
# r_squared is 0.239434  
# adjusted_r_squared is 0.236818

r_squared is 0.239434
adjusted_r_squared is 0.236819


## Level up (Optional)

- Perform variable selection using forward selection, using this resource: https://planspace.org/20150423-forward_selection_with_statsmodels/. Note that this time features are added based on the adjusted R-squared!
- Tweak the code in the `stepwise_selection()` function written above to just perform forward selection based on the p-value 

## Summary
Great! You practiced your feature selection skills by applying stepwise selection and recursive feature elimination to the Ames Housing dataset! 

In [14]:
import statsmodels.formula.api as smf

def forward_selected(data, response):
    """Linear model designed by forward selection.

    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response

    response: string, name of response column in data

    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by forward selection
           evaluated by adjusted R-squared
    """
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = 0.0, 0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(response,
                                           ' + '.join(selected + [candidate]))
            score = smf.ols(formula, data).fit().rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score < best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} + 1".format(response,
                                   ' + '.join(selected))
    model = smf.ols(formula, data).fit()
    return model

In [15]:
model_forward = forward_selected(preprocessed, 'SalePrice_log')

In [16]:
print (model_forward.model.formula)

SalePrice_log ~ GrLivArea_log + KitchenQual_TA + firstFlrSF_log + KitchenQual_Fa + Neighborhood_OldTown + KitchenQual_Gd + Neighborhood_Edwards + Neighborhood_IDOTRR + LotArea_log + Neighborhood_NridgHt + BldgType_Duplex + Neighborhood_NAmes + Neighborhood_SWISU + Neighborhood_BrkSide + Neighborhood_Sawyer + Neighborhood_NoRidge + Neighborhood_Somerst + Neighborhood_StoneBr + Neighborhood_MeadowV + SaleType_New + SaleType_WD + Neighborhood_BrDale + MSZoning_RM + MSZoning_RL + MSZoning_FV + MSZoning_RH + Neighborhood_NWAmes + Neighborhood_Mitchel + Neighborhood_SawyerW + SaleType_Con + Neighborhood_ClearCr + BldgType_2fmCon + SaleType_ConLD + Neighborhood_Veenker + Street_Pave + BldgType_Twnhs + Neighborhood_Timber + 1


In [17]:
print (model_forward.rsquared_adj)

0.8344715728761254


In [18]:
model_forward.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          SalePrice_log   R-squared:                       0.839
Model:                            OLS   Adj. R-squared:                  0.834
Method:                 Least Squares   F-statistic:                     199.8
Date:                Sat, 03 Oct 2020   Prob (F-statistic):               0.00
Time:                        21:08:07   Log-Likelihood:                -739.41
No. Observations:                1460   AIC:                             1555.
Df Residuals:                    1422   BIC:                             1756.
Df Model:                          37                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               -0.2021      0.230     -0.877      0.380      -0.654       0.250
GrLivArea_log            0.3775      0.015     24.817      0.000       0.348       0.407
KitchenQual_TA          -0.6710      0.055    -12.259      0.000      -0.778      -0.564
firstFlrSF_log           0.1389      0.015      9.181      0.000       0.109       0.169
KitchenQual_Fa          -1.0007      0.088    -11.426      0.000      -1.173      -0.829
Neighborhood_OldTown    -0.8396      0.065    -12.976      0.000      -0.967      -0.713
KitchenQual_Gd          -0.3838      0.050     -7.685      0.000      -0.482      -0.286
Neighborhood_Edwards    -0.6815      0.050    -13.762      0.000      -0.779      -0.584
Neighborhood_IDOTRR     -0.8642      0.097     -8.905      0.000      -1.055      -0.674
LotArea_log              0.1045      0.016      6.444      0.000       0.073       0.136
Neighborhood_NridgHt     0.4262      0.059      7.220      0.000       0.310       0.542
BldgType_Duplex         -0.4188      0.061     -6.842      0.000      -0.539      -0.299
Neighborhood_NAmes      -0.3599      0.040     -9.093      0.000      -0.438      -0.282
Neighborhood_SWISU      -0.6165      0.089     -6.902      0.000      -0.792      -0.441
Neighborhood_BrkSide    -0.5591      0.067     -8.372      0.000      -0.690      -0.428
Neighborhood_Sawyer     -0.3930      0.056     -7.023      0.000      -0.503      -0.283
Neighborhood_NoRidge     0.4411      0.071      6.232      0.000       0.302       0.580
Neighborhood_Somerst     0.1719      0.095      1.813      0.070      -0.014       0.358
Neighborhood_StoneBr     0.4817      0.087      5.514      0.000       0.310       0.653
Neighborhood_MeadowV    -0.6346      0.119     -5.325      0.000      -0.868      -0.401
SaleType_New             0.2767      0.069      3.996      0.000       0.141       0.413
SaleType_WD              0.1480      0.055      2.703      0.007       0.041       0.255
Neighborhood_BrDale     -0.4021      0.130     -3.098      0.002      -0.657      -0.148
MSZoning_RM              1.1074      0.151      7.327      0.000       0.811       1.404
MSZoning_RL              1.0147      0.160      6.332      0.000       0.700       1.329
MSZoning_FV              1.0747      0.192      5.596      0.000       0.698       1.451
MSZoning_RH              0.8904      0.193      4.616      0.000       0.512       1.269
Neighborhood_NWAmes     -0.1899      0.055     -3.457      0.001      -0.298      -0.082
Neighborhood_Mitchel    -0.1753      0.066     -2.668      0.008      -0.304      -0.046
Neighborhood_SawyerW    -0.1582      0.059     -2.682      0.007      -0.274      -0.043
SaleType_Con             0.5485      0.301      1.823      0.069      -0.042       1.139
Neighborhood_ClearCr    -0.1336      0.084     -1.593    